In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Bayesian Optimization


In [10]:
# Import the class
from xopt import Xopt
from botorch.test_functions.multi_fidelity import AugmentedHartmann

In [3]:
# To see log messages
from xopt import output_notebook
output_notebook()

The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [4]:
# Make a proper input file. 
YAML = """
xopt: {output_path: null}

algorithm:
  name: bayesian_optimization
  options:  
      n_initial_samples: 16
      n_steps: 6
      verbose: True
      generator_options:
          acquisition_function: custom_acq.acq
          use_gpu: False

simulation: 
  name: test_multi_fidelity
  evaluate: xopt.tests.evaluators.multi_fidelity.evaluate

vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = YAML

In [5]:
X = Xopt(config)
X

Loading config from text
Loading config from dict.
Loading config from dict.
Loading config from dict.
Loading config from dict.
`name` keyword no longer allowed in vocs config, removing
`description` keyword no longer allowed in vocs config, removing
`simulation` keyword no longer allowed in vocs config, removing
`templates` keyword no longer allowed in vocs config, moving to simulation `options`



            Xopt 
________________________________           
Version: 0.4.3+178.g392b747.dirty
Configured: True
Config as YAML:
xopt: {output_path: null}
algorithm:
  name: bayesian_optimization
  options:
    n_initial_samples: 16
    n_steps: 6
    verbose: true
    generator_options: {acquisition_function: !!python/name:custom_acq.acq '', use_gpu: false}
    custom_model: null
    restart_file: null
    initial_x: null
  function: xopt.bayesian.algorithms.bayesian_optimize
simulation:
  name: test_multi_fidelity
  evaluate: xopt.tests.evaluators.multi_fidelity.evaluate
  options: {templates: null, extra_option: abc}
  function: xopt.tests.evaluators.multi_fidelity.evaluate
vocs:
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  linked_variables: null
  constants: {a: dummy_constant}
  constraints: null

# Run BayesOpt

In [6]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

executor = PoolExecutor()
# This will also work. 
#executor=None

In [7]:
# Change max generations
X.run(executor=executor)
results = X.results

Starting at time 2021-09-27T22:15:26-07:00
started running optimization with generator: <xopt.bayesian.generators.generator.BayesianGenerator object at 0x7f81488d04c0>
submitting initial candidates at time 2021-09-27T22:15:26-07:00
starting optimization loop
submitting candidates at time 2021-09-27T22:15:27-07:00
submitting candidates at time 2021-09-27T22:15:28-07:00
submitting candidates at time 2021-09-27T22:15:28-07:00
submitting candidates at time 2021-09-27T22:15:29-07:00
submitting candidates at time 2021-09-27T22:15:30-07:00
submitting candidates at time 2021-09-27T22:15:30-07:00


### Get highest fidelity global optimum

In [14]:
# create generator object
from xopt.bayesian.generators.multi_fidelity import MultiFidelityGenerator
gen = MultiFidelityGenerator(X.vocs)

In [15]:
# create model
model = create_multi_fidelity_model(results['variables'], results['corrected_objectives'], results['corrected_constraints'], X.vocs)

NameError: name 'create_multi_fidelity_model' is not defined

In [ ]:
## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = gen.get_recommendation(model)
problem = AugmentedHartmann(negate=False)
problem(rec) ## NOTE: the correct global minimum is -3.32237